## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':3,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [ ]:
all_img_list = glob.glob('./custom_dataset/croped_imgs/*/*')

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])

In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.01, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [ ]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

## CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b7(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

## Inference

In [ ]:
import torchvision.transforms as transforms
def pre_image(image_path, model, crop):
   img = cv2.imread(image_path)
   img = img[crop[1]:crop[3], crop[0]:crop[2]]
#    cv2.imshow("Cars", img)
#    cv2.waitKey()
#    cv2.destroyAllWindows()
   mean = [0.485, 0.456, 0.406] 
   std = [0.229, 0.224, 0.225]
   transform_norm = transforms.Compose([transforms.ToTensor(), 
   transforms.Resize((224,224)),transforms.Normalize(mean, std)])
   # get normalized image
   img_normalized = transform_norm(img).float()
   img_normalized = img_normalized.unsqueeze_(0)
   # input = Variable(image_tensor)
   img_normalized = img_normalized.to(device)
   # print(img_normalized.shape)
   with torch.no_grad():
      model.eval()  
      output =model(img_normalized)
     # print(output)
      index = output.data.cpu().numpy().argmax()
      return index

pred = pd.read_csv('Results/3.csv')
results = pd.read_csv('./custom_dataset/sample_submission.csv')      
infer_model.eval()

for i in tqdm(range(len(pred))):
    scores = pred.iloc[i, :]
    crop = [int(scores[3]), int(scores[4]), int(scores[7]), int(scores[8])]
    image_path = os.getcwd()+'/custom_dataset/test/'+scores[0]
    output = pre_image(image_path, infer_model, crop)
    results = results.append({
            "file_name" : scores[0],               
            "class_id" : output,                                                        
            "confidence" : scores[2],                     
            "point1_x" : scores[3], "point1_y" : scores[4],
            "point2_x" : scores[5], "point2_y" : scores[6],
            "point3_x" : scores[7], "point3_y" : scores[8],
            "point4_x" : scores[9], "point4_y" : scores[10],
        }, ignore_index = True)

results.to_csv('cnn_submit.csv', index = False)

In [ ]:
import os
import cv2
pred = pd.read_csv('cnn_submit.csv')
img_path = './custom_dataset/test'
classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", 
               "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", 
               "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", 
               "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", 
               "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", 
               "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", 
               "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
               "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", 
               "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
               "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", 
               "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", 
               "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", 
               "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", 
               "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", 
               "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", 
               "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", 
               "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]

for i in range(len(pred)):
    scores = pred.iloc[i, :]
    label = "{}".format(classes[scores[1]]) + ' ' + str(scores[2])
    image_path = os.getcwd()+'/custom_dataset/test/'+scores[0]
    image = cv2.imread(image_path)
    cv2.rectangle(image, (int(scores[3]), int(scores[4])), (int(scores[7]), int(scores[8])), (0, 255, 0), 2)
    cv2.putText(image, label, (int(scores[3]), int(scores[4])-4), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        2 * 0.75, (0, 255, 0), 2, lineType=cv2.LINE_AA)
    cv2.imshow("Cars", image)
    cv2.waitKey()
    cv2.destroyAllWindows()